In [1]:
import scanpy as sc
import numpy as np
import pandas as pd

In [2]:
data_prefix = '/users/dgibbs/Work/CRUK/py-sc-niches/data/'

In [6]:
q = sc.read_h5ad(data_prefix+'../../gastric_fibs_and_epi0_4.h5ad')

In [7]:
q

AnnData object with n_obs × n_vars = 34883 × 35606
    obs: 'Cell_ID', 'samplename', 'Expressed Genes', 'n_molecules', 'doublet_score', 'Percent Mitochond.', 'leiden', 'Louvain Cluster', 'diagnosis', 'phase', 'sample_diagnosis', 'patient', 'nobatch_leiden', 'nobatch_louvain', 'clusters', 'diagnosis_class', 'Fb_Subtype', 'sample_name', 'major_cell_types', 'diagnosis_pathologist', 'diagnosis_major_celltype', 'diagnosis_minor_celltype', 'diagnosis_H&E', 'CancerType', 'Fine_Grained_Cell_Labels'

In [37]:
q.obs.major_cell_types.value_counts()

major_cell_types
surface_cell                 9941
IM_cell                      5790
isthmic_progenitor           5532
spem_mucus_neck_cells        2874
chief_cell                   2368
enteroendocrine_cell         2106
dysplastic_cancer            1744
other                        1101
transdifferentiating_cell     384
parietal_cell                 334
Name: count, dtype: int64

In [39]:
def subsample_balanced(adata, group_key, n_per_group=None, random_state=42):
    """Subsample an AnnData object so that each group in `group_key` has equal cell count."""
    groups = adata.obs[group_key]
    rng = np.random.default_rng(seed=random_state)

    # Determine number to sample per group
    group_sizes = groups.value_counts()
    min_group_size = group_sizes.min()
    n = min_group_size if n_per_group is None else min(n_per_group, min_group_size)

    # Sample indices per group
    sampled_indices = []
    for g in group_sizes.index:
        idx = adata.obs[adata.obs[group_key] == g].index
        sampled = rng.choice(idx, size=n, replace=False)
        sampled_indices.extend(sampled)

    # Return new AnnData object
    return adata[sampled_indices].copy()

qsamp = subsample_balanced(q, 'major_cell_types', n_per_group=100)

In [40]:
qsamp.obs.major_cell_types.value_counts()

major_cell_types
IM_cell                      100
chief_cell                   100
dysplastic_cancer            100
enteroendocrine_cell         100
isthmic_progenitor           100
other                        100
parietal_cell                100
spem_mucus_neck_cells        100
surface_cell                 100
transdifferentiating_cell    100
Name: count, dtype: int64

In [8]:
q.obs

,Cell_ID,samplename,Expressed Genes,n_molecules,doublet_score,Percent Mitochond.,leiden,Louvain Cluster,diagnosis,phase,...,diagnosis_class,Fb_Subtype,sample_name,major_cell_types,diagnosis_pathologist,diagnosis_major_celltype,diagnosis_minor_celltype,diagnosis_H&E,CancerType,Fine_Grained_Cell_Labels
0,AACACACCATTGCTGA-G08B,G08B,6544,25638.0,0.083463,0.022974,4,4,Tumor,G1,...,dysplasia_tumor,FbS1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AACCATGTCCGCTTAC-G08B,G08B,1717,2775.0,0.077979,0.002883,11,3,Tumor,G1,...,dysplasia_tumor,FbS1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AATGAAGAGAAGTCAT-G08B,G08B,1512,3048.0,0.096688,0.010827,21,3,Tumor,G1,...,dysplasia_tumor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ACTTTGTTCTCTAGGA-G08B,G08B,3822,11907.0,0.125093,0.001596,10,6,Tumor,G1,...,dysplasia_tumor,FbS1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGCCAATAGGGAACAA-G08B,G08B,4281,14539.0,0.089651,0.002476,16,6,Tumor,G1,...,dysplasia_tumor,FbS1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32169,32169,NaN,6797,26104.0,0.088235,0.130248,12,13,NaN,G1,...,NaN,NaN,G06A,enteroendocrine_cell,Metaplasia/Dysplasia,IM,SPEM,IM,intestinal-type,"Enteroendocrine (ghrelin, SST, gastrin) cells"
32170,32170,NaN,1175,3131.0,0.026465,0.184606,0,0,NaN,G1,...,NaN,NaN,G06A,surface_cell,Metaplasia/Dysplasia,IM,SPEM,IM,intestinal-type,Surface cells
32171,32171,NaN,1038,1493.0,0.037244,0.008707,5,7,NaN,G2M,...,NaN,NaN,G06A,IM_cell,Metaplasia/Dysplasia,IM,SPEM,IM,intestinal-type,Intestinal metaplasia (IM) Intermediate cells ...
32172,32172,NaN,7393,52382.0,0.088235,0.131133,11,12,NaN,G1,...,NaN,NaN,G06A,dysplastic_cancer,Metaplasia/Dysplasia,IM,SPEM,IM,intestinal-type,Dysplastic cells (TACSTD2+ CEACAMS+)


In [9]:
q.var.head()

""
A1BG
A1CF
A2M
A2ML1
A2MP1


In [7]:
x1 = q[:,"IL10"].X.todense()
x2 = q[:,"IL10RA"].X.todense()

In [24]:
genes = np.random.choice(q.var.index.values, size=10, replace=False)
ci = 20
cj = 1000

In [25]:
genes

array(['ENST00000604983', 'FASTKD1', 'PALD1', 'SERINC1', 'HLA-H',
       'ENST00000558917', 'MYL12BP1', 'CR2', 'ARG1', 'PPA1'], dtype=object)

In [26]:
q[ci, genes].X.todense()

matrix([[0.       , 0.7570596, 0.       , 1.514119 , 0.       ,
         0.       , 0.       , 0.       , 0.       , 1.514119 ]],
       dtype=float32)

In [27]:
q[ci,"SDHAP1"].X.todense()

matrix([[1.514119]], dtype=float32)

In [28]:
q.obs.iloc[ci]

Cell_ID                     CTCACTGGTACAATAG-G08B
samplename                                   G08B
Expressed Genes                              4323
n_molecules                               13802.0
doublet_score                            0.068692
Percent Mitochond.                       0.005362
leiden                                          4
Louvain Cluster                                 4
diagnosis                                   Tumor
phase                                          G1
sample_diagnosis                       G08B_Tumor
patient                                       G08
nobatch_leiden                                  0
nobatch_louvain                                 2
clusters                                   PDGFRA
diagnosis_class                   dysplasia_tumor
Fb_Subtype                                   FbS1
sample_name                                   NaN
major_cell_types                              NaN
diagnosis_pathologist                         NaN


In [29]:
q.obs.iloc[cj]

Cell_ID                     CATTGCCGTCCACACG-G07A
samplename                                   G07A
Expressed Genes                              2118
n_molecules                                4786.0
doublet_score                            0.064011
Percent Mitochond.                        0.06122
leiden                                          1
Louvain Cluster                                 1
diagnosis                        Normal (Stomach)
phase                                          G1
sample_diagnosis            G07A_Normal (Stomach)
patient                                       G07
nobatch_leiden                                  3
nobatch_louvain                                 4
clusters                                    FBLN2
diagnosis_class                   adjacent_normal
Fb_Subtype                                   FbS2
sample_name                                   NaN
major_cell_types                              NaN
diagnosis_pathologist                         NaN


In [30]:
x1 = q[ci,genes].X.todense()
x2 = q[cj,genes].X.todense()

In [31]:
x1

matrix([[0.       , 0.7570596, 0.       , 1.514119 , 0.       ,
         0.       , 0.       , 0.       , 0.       , 1.514119 ]],
       dtype=float32)

In [32]:
x2

matrix([[0.      , 0.      , 0.      , 4.476276, 0.      , 0.      ,
         0.      , 0.      , 0.      , 0.      ]], dtype=float32)

In [33]:
np.multiply(x1, x2)

matrix([[0.       , 0.       , 0.       , 6.7776146, 0.       ,
         0.       , 0.       , 0.       , 0.       , 0.       ]],
       dtype=float32)

In [10]:
merged_df = pd.read_csv(data_prefix+'omnipath_ligrecextra_annot.csv')
#/users/dgibbs/Work/CRUK/py-sc-niches/data/omnipath_ligrecextra_annot.csv

In [52]:
merged_df

,row,source_protein,target_protein,source_ensg,target_ensg,source_symbol,target_symbol
0,0,P0DP23,Q13507,ENSG00000198668,ENSG00000138741,CALM1,TRPC3
1,1,P46531,Q9Y219,ENSG00000148400,ENSG00000184916,NOTCH1,JAG2
2,2,Q9Y219,P46531,ENSG00000184916,ENSG00000148400,JAG2,NOTCH1
3,3,O00548,P46531,ENSG00000275555,ENSG00000148400,DLL1,NOTCH1
4,4,O00548,P46531,ENSG00000198719,ENSG00000148400,DLL1,NOTCH1
...,...,...,...,...,...,...,...
6648,6648,P17936,Q6UXB1,ENSG00000146674,ENSG00000188624,IGFBP3,IGFL3
6649,6649,P18065,Q6UXB1,ENSG00000115457,ENSG00000188624,IGFBP2,IGFL3
6650,6650,O75487,P01308,ENSG00000076716,ENSG00000254647,GPC4,INS
6651,6651,Q969E1,Q9UBU3,ENSG00000164406,ENSG00000157017,LEAP2,GHRL


In [11]:
filtered_df = merged_df[
    merged_df["source_symbol"].isin(q.var.index) &
    merged_df["target_symbol"].isin(q.var.index)
]

In [68]:
filtered_df.shape

(6598, 7)

In [12]:
source_genes = filtered_df.source_symbol.values
target_genes = filtered_df.target_symbol.values

source_target = [x+'_'+y for x,y in zip(source_genes, target_genes)]

In [36]:
seen_pairs = set()
unique_source_target = []
unique_source_genes = []
unique_target_genes = []

for pair, s_genes, t_genes in zip(source_target, source_genes, target_genes):
    if pair not in seen_pairs:
        seen_pairs.add(pair)
        unique_source_target.append(pair)
        unique_source_genes.append(s_genes)
        unique_target_genes.append(t_genes)

In [73]:
# if we have two cells, ci and cj,
# and a list of genes, sources and targets,
# then we just have to extract them and multiple

np.all( [x in q.var.index for x in source_genes] )

#true

True

In [118]:
q.shape[1]

35606

In [115]:
x1 = q[ci,source_genes].X.todense()
x2 = q[cj,target_genes].X.todense()
x1_x2 = np.multiply(x1,x2)

In [116]:
np.sum( x1_x2 > 0 )

134

In [112]:
cellid = str(ci)+'_'+str(cj)

cols_ci = ['ci_'+colx for colx in q.obs.columns]
cols_cj = ['cj_'+colx for colx in q.obs.columns]

df_ci = q.obs.iloc[[ci]].copy()  # double brackets -> DataFrame
df_cj = q.obs.iloc[[cj]].copy()

df_ci.columns = cols_ci
df_cj.columns = cols_cj

# Concatenate side-by-side → single row
df_obs_merge = pd.concat([df_ci.reset_index(drop=True), df_cj.reset_index(drop=True)], axis=1)
# Add identifier
df_obs_merge["cellcellid"] = cellid

In [114]:
df_obs_merge

,ci_Cell_ID,ci_samplename,ci_Expressed Genes,ci_n_molecules,ci_doublet_score,ci_Percent Mitochond.,ci_leiden,ci_Louvain Cluster,ci_diagnosis,ci_phase,...,cj_Fb_Subtype,cj_sample_name,cj_major_cell_types,cj_diagnosis_pathologist,cj_diagnosis_major_celltype,cj_diagnosis_minor_celltype,cj_diagnosis_H&E,cj_CancerType,cj_Fine_Grained_Cell_Labels,cellcellid
0,CTCACTGGTACAATAG-G08B,G08B,4323,13802.0,0.068692,0.005362,4,4,Tumor,G1,...,FbS2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20_1000


In [13]:
qmini = sc.pp.subsample(data=q, fraction=0.005, copy=True)

qmini.shape

(174, 35606)

In [ ]:
# save up the rows of the cell-pair obs
obs_list = []
X_list = []
error_list = []

qq = qmini.copy()

### for each pair of cells:
for ci in range(qq.shape[0]):
    for cj in range(ci+1,qq.shape[0]):

        try:
            # the name of the cell pairing
            cellid = str(ci)+'_'+str(cj)
            df_ci = qq.obs.iloc[[ci]].copy()  # double brackets -> DataFrame
            df_cj = qq.obs.iloc[[cj]].copy()
            df_ci.columns = cols_ci
            df_cj.columns = cols_cj

            # Concatenate side-by-side → single row
            df_obs_merge = pd.concat([df_ci.reset_index(drop=True), df_cj.reset_index(drop=True)], axis=1)

            # Add identifier
            df_obs_merge["cellcellid"] = cellid

            # get the count vectors
            x1 = qq[ci,source_genes].X.todense()
            x2 = qq[cj,target_genes].X.todense()

            # pairwise dot product
            x1_x2 = np.multiply(x1,x2)

            obs_list.append(df_obs_merge)
            X_list.append(x1_x2)
        except:
            print("error on cell pair:" + str(cellid))
            error_list.append(cellid)



error on cell pair:45_156
error on cell pair:47_104
error on cell pair:47_123
error on cell pair:47_141
error on cell pair:47_159
error on cell pair:47_177
error on cell pair:47_192
error on cell pair:47_205
error on cell pair:80_541
error on cell pair:80_559
error on cell pair:80_580
error on cell pair:80_598
error on cell pair:80_616
error on cell pair:80_638
error on cell pair:81_110
error on cell pair:81_128
error on cell pair:81_146
error on cell pair:81_165
error on cell pair:99_424
error on cell pair:99_440
error on cell pair:99_455


In [20]:
from joblib import Parallel, delayed
import numpy as np
import pandas as pd

# Chunking utility
def chunk_indices(n, chunk_size):
    return [list(range(i, min(i + chunk_size, n))) for i in range(0, n, chunk_size)]

# Core logic to process a chunk of ci indices
def process_ci_chunk(ci_chunk, qq, source_genes, target_genes, cols_ci, cols_cj):
    obs_list = []
    X_list = []
    error_list = []

    print(ci_chunk)

    for ci in ci_chunk:
        for cj in range(ci + 1, qq.shape[0]):
            try:
                cellid = f"{ci}_{cj}"
                df_ci = qq.obs.iloc[[ci]].copy()
                df_cj = qq.obs.iloc[[cj]].copy()
                df_ci.columns = cols_ci
                df_cj.columns = cols_cj

                df_obs_merge = pd.concat([df_ci.reset_index(drop=True),
                                          df_cj.reset_index(drop=True)], axis=1)
                df_obs_merge["cellcellid"] = cellid

                x1 = qq[ci, source_genes].X.todense()
                x2 = qq[cj, target_genes].X.todense()
                x1_x2 = np.multiply(x1, x2)

                obs_list.append(df_obs_merge)
                X_list.append(x1_x2)
            except:
                error_list.append(f"{ci}_{cj}")

    return obs_list, X_list, error_list


In [21]:
qq = sc.pp.subsample(data=q, fraction=0.005, copy=True)
print(qq.shape)
n_cells = qq.shape[0]
chunk_size = 100  # adjust based on memory and CPU cores
ci_chunks = chunk_indices(n_cells, chunk_size)

cols_ci = ['ci_'+colx for colx in q.obs.columns]
cols_cj = ['cj_'+colx for colx in q.obs.columns]

results = Parallel(n_jobs=8, backend="loky")(
    delayed(process_ci_chunk)(ci_chunk, qq, source_genes, target_genes, cols_ci, cols_cj)
    for ci_chunk in ci_chunks
)

# Merge results
obs_list   = [item for chunk in results for item in chunk[0]]
X_list     = [item for chunk in results for item in chunk[1]]
error_list = [item for chunk in results for item in chunk[2]]

(174, 35606)


In [22]:
obs_df = pd.concat(obs_list, axis=0)

In [26]:
obs_df

,ci_Cell_ID,ci_samplename,ci_Expressed Genes,ci_n_molecules,ci_doublet_score,ci_Percent Mitochond.,ci_leiden,ci_Louvain Cluster,ci_diagnosis,ci_phase,...,cj_Fb_Subtype,cj_sample_name,cj_major_cell_types,cj_diagnosis_pathologist,cj_diagnosis_major_celltype,cj_diagnosis_minor_celltype,cj_diagnosis_H&E,cj_CancerType,cj_Fine_Grained_Cell_Labels,cellcellid
0,21031,NaN,3846,15893.0,0.045336,0.085572,4,3,NaN,G1,...,NaN,G09D,surface_cell,Metaplasia,IM,"Dys, SPEM",Dys/Can,diffuse-type,Surface cells,0_1
0,21031,NaN,3846,15893.0,0.045336,0.085572,4,3,NaN,G1,...,NaN,G07B,IM_cell,Tumor,IM,Dys,Dys/Can,diffuse-type,IM intermediate cells (DMBT1+),0_2
0,21031,NaN,3846,15893.0,0.045336,0.085572,4,3,NaN,G1,...,NaN,G07B,dysplastic_cancer,Tumor,IM,Dys,Dys/Can,diffuse-type,Dysplastic cells (TACSTD2+ CEACAMS+),0_3
0,21031,NaN,3846,15893.0,0.045336,0.085572,4,3,NaN,G1,...,NaN,G08C,surface_cell,Metaplasia,"SPEM, Surface",NaN,SPEM,intestinal-type,Surface cells,0_4
0,21031,NaN,3846,15893.0,0.045336,0.085572,4,3,NaN,G1,...,NaN,G07D,isthmic_progenitor,Dysplasia,SPEM,"IM, Dys",Dys/Can,diffuse-type,Commited progenitors,0_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,10833,NaN,634,1050.0,0.016578,0.107619,8,5,NaN,G1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,170_172
0,10833,NaN,634,1050.0,0.016578,0.107619,8,5,NaN,G1,...,NaN,G07A,transdifferentiating_cell,Normal,"SPEM, Chief, Surface",NaN,Normal,diffuse-type,Transdifferentiating cells,170_173
0,706,NaN,7458,53407.0,0.058842,0.116576,24,24,NaN,G2M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,171_172
0,706,NaN,7458,53407.0,0.058842,0.116576,24,24,NaN,G2M,...,NaN,G07A,transdifferentiating_cell,Normal,"SPEM, Chief, Surface",NaN,Normal,diffuse-type,Transdifferentiating cells,171_173


In [27]:
from scipy.sparse import csr_matrix, vstack

# Convert each dense matrix (1 x N) to a sparse row
X_sparse_rows = [csr_matrix(x) for x in X_list]

# Stack into a single sparse matrix
X_sparse = vstack(X_sparse_rows, format='csr')

In [28]:
X_sparse.shape

(15051, 6598)

In [30]:
var_df = pd.DataFrame({'source_symbol': source_genes,
                       'target_symbol': target_genes}, index=source_target)

qfinal = sc.AnnData(X=X_sparse, obs=obs_df, var=var_df)

/users/dgibbs/miniconda3/envs/ScanpySquidpy/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/users/dgibbs/miniconda3/envs/ScanpySquidpy/lib/python3.11/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/users/dgibbs/miniconda3/envs/ScanpySquidpy/lib/python3.11/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [31]:
qfinal

AnnData object with n_obs × n_vars = 15051 × 6598
    obs: 'ci_Cell_ID', 'ci_samplename', 'ci_Expressed Genes', 'ci_n_molecules', 'ci_doublet_score', 'ci_Percent Mitochond.', 'ci_leiden', 'ci_Louvain Cluster', 'ci_diagnosis', 'ci_phase', 'ci_sample_diagnosis', 'ci_patient', 'ci_nobatch_leiden', 'ci_nobatch_louvain', 'ci_clusters', 'ci_diagnosis_class', 'ci_Fb_Subtype', 'ci_sample_name', 'ci_major_cell_types', 'ci_diagnosis_pathologist', 'ci_diagnosis_major_celltype', 'ci_diagnosis_minor_celltype', 'ci_diagnosis_H&E', 'ci_CancerType', 'ci_Fine_Grained_Cell_Labels', 'cj_Cell_ID', 'cj_samplename', 'cj_Expressed Genes', 'cj_n_molecules', 'cj_doublet_score', 'cj_Percent Mitochond.', 'cj_leiden', 'cj_Louvain Cluster', 'cj_diagnosis', 'cj_phase', 'cj_sample_diagnosis', 'cj_patient', 'cj_nobatch_leiden', 'cj_nobatch_louvain', 'cj_clusters', 'cj_diagnosis_class', 'cj_Fb_Subtype', 'cj_sample_name', 'cj_major_cell_types', 'cj_diagnosis_pathologist', 'cj_diagnosis_major_celltype', 'cj_diagnosis_mi

In [32]:
qfinal.var

,source_symbol,target_symbol
CALM1_TRPC3,CALM1,TRPC3
NOTCH1_JAG2,NOTCH1,JAG2
JAG2_NOTCH1,JAG2,NOTCH1
DLL1_NOTCH1,DLL1,NOTCH1
DLL1_NOTCH1,DLL1,NOTCH1
...,...,...
IGFBP3_IGFL3,IGFBP3,IGFL3
IGFBP2_IGFL3,IGFBP2,IGFL3
GPC4_INS,GPC4,INS
LEAP2_GHRL,LEAP2,GHRL
